In [1]:
from openai import OpenAI
import pandas as pd

from dotenv import load_dotenv
import os
import numpy as np
from tqdm import tqdm

import faiss

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [ ]:
report = pd.read_csv("./data/csv/report.csv")
report = report.fillna('')
report.isnull().sum()

image_name       0
image_path       0
findings         0
impression       0
findings_jp      0
impression_jp    0
dtype: int64

In [ ]:
for index, row in tqdm(report.iterrows(), total=report.shape[0]):
    if index < 1800:
        continue
    input = row['findings_jp'] + row['impression_jp']
    res = client.embeddings.create(
        model="text-embedding-3-large",
        input=input
    )
    vector = res.data[0].embedding
    image_name = row.image_name
    name, ext = os.path.splitext(image_name)

    vector = np.array(vector)
    np.save("./data/emb/" + name + ".npy", vector)

100%|██████████| 10000/10000 [58:21<00:00,  2.86it/s] 


In [ ]:
concat_emb = []
for index, row in tqdm(report.iterrows(), total=report.shape[0]):
    image_name = row.image_name
    name, ext = os.path.splitext(image_name)
    emb = np.load("./data/emb/" + name + ".npy")
    concat_emb.append(emb)

concat_emb = np.array(concat_emb)
np.save("./data/emb/all_emb.npy", concat_emb)

100%|██████████| 10000/10000 [00:06<00:00, 1548.10it/s]


In [ ]:
all_emb = np.load("./data/emb/all_emb.npy")
index = faiss.IndexFlatIP(all_emb.shape[1])
index.add(all_emb)

faiss.write_index(index, "./data/faiss/faiss_index.idx")